<a href="https://colab.research.google.com/github/Baldezo313/NPL_Text_Generation/blob/main/src/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Génération de texte avec les réseaux de neurones  

Dans ce projet, nous allons créer un réseau qui peut générer du texte, ici nous montrons que cela se fait caractère par caractère.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Étape 1 : Les Données  

Vous pouvez récupérer n'importe quel texte gratuit ici : https://www.gutenberg.org/

Nous allons choisir toutes les œuvres de Shakespeare, principalement pour deux raisons :

1. C'est un grand corpus de textes, il est généralement recommandé d'avoir au moins une source d'un million de caractères au total pour obtenir une génération de texte réaliste.

2. Il a un style très particulier. Comme les données textuelles utilisent un anglais ancien et sont formatées dans le style d'une pièce de théâtre, il nous apparaîtra très clairement si le modèle est capable de reproduire des résultats similaires.

In [3]:
path_to_file = "/content/sample_data/shakespeare.txt"

In [4]:
text = open(path_to_file, 'r').read()

In [5]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


* **Comprendre les caractères uniques**

In [6]:
# Les caractères uniques dans le fichier
vocab = sorted(set(text))
print(vocab)
len(vocab)


['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Etape 2: Traitement du Texte  
* **Vectorisation du texte:**  
  
Nous savons qu'un réseau de neurones ne peut pas prendre en charge les données brutes des chaînes de caractères, nous deveons attribuer des numéros à chaque caractères. Créons deux dictionnaires qui peuvent passer d'un index numérique à un caractères et d'un caractère à un index numérique.

In [7]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [8]:
ind_to_char = np.array(vocab)
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [9]:
encoded_text = np.array([char_to_ind[c] for c in text])
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [10]:
encoded_text.shape

(5445609,)

  Nous disposons maintenant d'un mapping que nous pouvons utiliser pour passer des caractères aux chiffres.

In [11]:
sample = text[:20]
sample

'\n                   '

In [12]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Étape 3 : Création de batches  

Globalement, ce que nous essayons de faire, c'est de faire en  sorte que le modèle prévoie le caractère suivant le plus probable, compte tenu d'une séquence historique de caractères. C'est à nous (l'utisateur) de choisir la longueur de cette séquence historique. Une séquence trop courte et nous n'aurons pas assez d'informations (par exemple, étant donné la lettre "a", quel est le ,prochaine caractère), une séquence trop longue et l'entrainement prendra trop de temps et risque de sur-entrainer (au riques d'obtenir une séquence de caractère qui ne sont pas pertinents pour des caractères plus éloignés). Bien qu'il n'y ait pas de choix correct de longueur de séquence, on dois considerer le texte lui-même, la longueur des phrases normales qu'il contient et avoir une idée raisonnable des caractères/mots qui ne sont pertinents les uns pour les autres.

In [13]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [14]:
line = "From fairest creatures we desire increase,"
len(line)

42

In [15]:
part_stanza = '''From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,'''

len(part_stanza)

131

#### Sequences d'Entraînement
Les données textuelles réelles seront la séquence de texte décalée d'un caractère vers l'avant.  

Par exemple:  
- Sequence In: "Hello my name"  
- Sequence Out: "ello my name"  

Nous pouvons utiliser la fonction `tf.data.Dataset.from_tensor_slices` pour convertir un vecteur de texte en un flux d'indices de caractères.

In [16]:
seq_len = 120

In [17]:
total_num_seq = len(text) // (seq_len + 1)
total_num_seq

45005

In [18]:
# Création des séquences d'entrainement
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for item in char_dataset.take(500):
    print(ind_to_char[item.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


La méthode du **batch** convertit ces appels de caractères individuels en séquences que nous pouvons alimenter en lot. Nous utilions seq_len+1 en raison de l'indexation zéro.  

Voici ce que signifie drop_remainder:  

*drop_remainder*: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing whether the last batch should be dropped in the case it has fewer than `batch_size` elements; the default behavior is not to drop the smaller batch.   

In [19]:
type(char_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [20]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Maintenant que nous avons nos séquences, nous allons effectuer les étapes suivantes pour chacune d'entre elles afin de créer nos séquences de texte cible.  
1. Saisir la séquence de texte d'entrée  
2. Assigner la séquence de texte cible comme séquence de texte d'entrée décalée d'un pas en avant.  
3. Regroupez-les en un tuple  


In [21]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [22]:
dataset = sequences.map(create_seq_targets)

In [23]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print(target_txt.numpy())
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


#### Générer des batches d'entrainement  

Maintenant que nous avons les séquences réelles, nous allons créer les lots, nous voulons mélanger ces séquences dans un ordre aléatoire, de sorte que le modèle ne s'adapte à aucune section du texte, mais puisse au contraire générer des caractère à partir de n'importe quel texte de départ.  


In [24]:
# Taille de batch
batch_size = 128

# Taille de la mémoire tampon pour mélanger l'ensemble des données
# afin de ne pas tenter de mélanger toute la séquence en mémoire.
# Au lieu de cela, il maintient une mémoire tampon dans laquelle
# il mélange les éléments
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [25]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

## Etape 4: Création du modèle  

Nous utiliserons un modèle basé sur le LSTM avec quelques caractéristiques supplémentaires, notamment une couche embedding pour commencer et deux couches LSTM. Nous avons basé cette architecture de modèle sur [DeepMoji](https://deepmoji.mit.edu/) et le code source original peut être trouvé [ici](https://github.com/bfelbo/DeepMoji).

La couche embedding servira de couche d'entrée, qui crée essentiellement une table de consultation qui fait correspondre les indices numériques de chaque caractères à un vecteur avec un nombre de dimensions "embedding dim". Comme on peut l'imaginer, plus cette taille d'embedding est grande, plus l'entraînement est complexe. C'est similaire à l'idée derrière word2vec, oû les mots sont mis en correspondance avec un espace à n dimensions. L'embedding avant le feeding direct dans le LSTM conduit généralement à des résultats plus réalistes.

In [26]:
# Longueur du vocabulaire en caractères
vocab_size = len(vocab)

# La dimension embedding
embed_dim = 64

# Nombre d'unitées RNN
rnn_neurones = 1026

Créons maintenant une fonction qui s'adapte facilement aux différentes variables comme indiqué ci-dessus.

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, GRU, Dense

#### Mise en place de la fonction de perte  

Pour notre part, nous utiliserons une crossentropie catégorique peu dense (sparse_categorical_crossentropy), que nous pouvons importer de Keras. Nous définirons également ce paramètre `logits=True`

In [28]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [29]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module keras.src.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1, ignore_class=None)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    >>> y_true = [[[ 0,  2],
    ...            [-1, -1]],
    ...           [[ 0,  2],
    ...            [-1, -1]]]
    >>> y_pred = [[[[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]],
    ...             [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]],
    ...           [[[1.0, 0.0, 0.0], [0.0, 0.5, 0.5]],
    ...            [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(
    ...   y_true, y_pred, ignore_class=-1)
    >>> loss.numpy()
    array([[[2

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [30]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [31]:
def create_model(vocab_size, embed_dim, rnn_neurones, batch_size):
    model = Sequential()

    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))

    model.add(GRU(rnn_neurones, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))

    # Couche Finale Dense de Prédiction
    model.add(Dense(vocab_size))

    model.compile(optimizer='adam', loss=sparse_cat_loss)

    return model

In [32]:
model = create_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    rnn_neurones=rnn_neurones,
    batch_size=batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Etape 5: Entrainement du modèle  

Assurons-nous que tout va bien avec notre modèle avant de passer trop de temps sur l'entrainement! Passons en lot pour confirmer que le modèle prédit actuellement des caractères aléatoires sans aucun entraînement.  

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    # Prédire sur un lot aléatoire
    example_batch_predictions = model(input_example_batch)

    # Afficher les dimensions des prédictions
    print(example_batch_predictions.shape, "<=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84) <=== (batch_size, sequence_length, vocab_size)


In [34]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 6.45444321e-04,  5.04791038e-04,  9.46331071e-04, ...,
         -2.29320372e-03, -1.50430982e-03, -1.02553349e-02],
        [ 1.56125217e-03,  1.53367326e-03,  5.93342469e-04, ...,
         -3.12202028e-03, -3.58828134e-03, -1.59327406e-02],
        [ 2.18758220e-03,  2.42294162e-03, -7.60422263e-05, ...,
         -3.29200062e-03, -5.17784804e-03, -1.90665368e-02],
        ...,
        [ 5.17163286e-03,  3.48935835e-03,  2.21086317e-03, ...,
         -6.99123787e-03, -9.41390730e-03, -6.25078240e-03],
        [ 3.09444987e-03, -3.69275780e-03, -5.27907454e-04, ...,
         -8.26895994e-04, -6.68878993e-03, -4.17404063e-03],
        [-9.83467442e-04, -1.68880937e-03,  6.37631537e-03, ...,
         -6.40799699e-04, -4.49449569e-03, -4.07541869e-03]],

       [[ 6.45444321e-04,  5.04791038e-04,  9.46331071e-04, ...,
         -2.29320372e-03, -1.50430982e-03, -1.02553349e-02],
        [ 1.56125217e-03,  1.53367326e-03,  5.9

In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [36]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[44],
       [40],
       [25],
       [80],
       [50],
       [30],
       [26],
       [ 9],
       [65],
       [30],
       [23],
       [27],
       [67],
       [40],
       [76],
       [48],
       [39],
       [ 9],
       [17],
       [ 8],
       [38],
       [21],
       [36],
       [48],
       [52],
       [43],
       [68],
       [59],
       [48],
       [ 5],
       [ 1],
       [81],
       [68],
       [76],
       [47],
       [40],
       [76],
       [60],
       [47],
       [ 0],
       [ 5],
       [81],
       [ 5],
       [17],
       [55],
       [29],
       [42],
       [ 5],
       [63],
       [64],
       [54],
       [20],
       [17],
       [30],
       [62],
       [ 8],
       [24],
       [31],
       [62],
       [74],
       [ 9],
       [47],
       [13],
       [44],
       [63],
       [42],
       [50],
       [58],
       [60],
       [17],
       [75],
       [40],
       [21],
   

In [37]:
# Reformater pour ne pas être une liste de listes
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

In [38]:
sampled_indices

array([44, 40, 25, 80, 50, 30, 26,  9, 65, 30, 23, 27, 67, 40, 76, 48, 39,
        9, 17,  8, 38, 21, 36, 48, 52, 43, 68, 59, 48,  5,  1, 81, 68, 76,
       47, 40, 76, 60, 47,  0,  5, 81,  5, 17, 55, 29, 42,  5, 63, 64, 54,
       20, 17, 30, 62,  8, 24, 31, 62, 74,  9, 47, 13, 44, 63, 42, 50, 58,
       60, 17, 75, 40, 21, 78, 77, 18, 78, 14, 38, 46, 61, 44, 34, 35, 30,
        1, 82,  6, 29, 22, 80, 39, 19, 59, 53, 39,  8, 35, 30, 52, 48, 81,
       81, 58,  8, 56, 36, 83, 41, 63, 28,  2, 26, 48, 60, 27, 71, 46, 76,
       62])

In [39]:
print("Compte tenu de la séquence d'entrée : \n")
print("".join(ind_to_char[input_example_batch[0]]))
print("\n")
print("Prochain caractère prédit : \n")
print("".join(ind_to_char[sampled_indices]))

Compte tenu de la séquence d'entrée : 

    [Puts down the skull.]
  Hor. E'en so, my lord.
  Ham. To what base uses we may return, Horatio! Why may not
    ima


Prochain caractère prédit : 

SO?yYEA-jE<BlOuWN-6,M:KW[RmdW' zmuVOueV
'z'6`DQ'hi_96Eg,>Fgs-V2ShQYce6tO:wv7w3MUfSIJE |(D;yN8d]N,JE[Wzzc,aK}PhC!AWeBpUug


Après avoir confirmé que les dimensions fonctionnent, entraînons notre réseau !

In [40]:
epochs = 30


In [41]:
model.fit(dataset, epochs=epochs)

Epoch 1/30
351/351 [==============================] - 49s 125ms/step - loss: 2.5182
Epoch 2/30
351/351 [==============================] - 46s 122ms/step - loss: 1.7117
Epoch 3/30
351/351 [==============================] - 47s 129ms/step - loss: 1.4497
Epoch 4/30
351/351 [==============================] - 48s 132ms/step - loss: 1.3347
Epoch 5/30
351/351 [==============================] - 48s 131ms/step - loss: 1.2739
Epoch 6/30
351/351 [==============================] - 50s 136ms/step - loss: 1.2344
Epoch 7/30
351/351 [==============================] - 51s 139ms/step - loss: 1.2060
Epoch 8/30
351/351 [==============================] - 49s 134ms/step - loss: 1.1825
Epoch 9/30
351/351 [==============================] - 50s 138ms/step - loss: 1.1627
Epoch 10/30
351/351 [==============================] - 49s 133ms/step - loss: 1.1452
Epoch 11/30
351/351 [==============================] - 48s 131ms/step - loss: 1.1295
Epoch 12/30
351/351 [==============================] - 51s 136ms/step - lo

## Etape 6: Génération du texte

Actuellement, notre modéle ne prévoit que 128 séquences à la fois. Nous pouvons créer un nouveau modèle qui n'attend qu'un batch_size=1.  
Nous pouvons créer un nouveau modèle avec cette taille de batch, puis charger les poids de nos modèles sauvegardés. Ensuite, appelez `.build()` sur le model:

In [42]:
model.save("shakespeare_gen.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
from tensorflow.keras.models import load_model

In [44]:
model = create_model(vocab_size, embed_dim, rnn_neurones, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
def generate_text(model, start_seed, gen_size=100, temp=1.0):
  '''
  model: Modèle Entraîné pour générer du texte
  start_seed: Seed initial du texte sous forme de chaîne de caractères
  gen_size: Nombre de caractères à générer

  L'idée de base de cette fonction est de prendre un texte de départ, de le formater de manière à
  q'il soit dans le bon format pour notre réseau, puis bouclez la séquence à mesure que nous
  continuons d'ajouter nos propres caractères prédits. Similaire à notre travail au sein des problèmes
  de séries temporelles avec le RNN.
  '''

  # Nombre de caractères à générer
  num_generate = gen_size

  # Vectorisation du texte du seed de départ
  input_eval = [char_to_ind[s] for s in start_seed]

  # Etendre les dimensions pour correspondre à la forme du format de batch
  input_eval = tf.expand_dims(input_eval, 0)

  # Liste vide pour contenir le texte généré
  text_generated = []

  # La température a un effet aléatoire sur le texte qui en résulte
  # Le terme est dérivé de l'entropie/thermodynamique.
  # La temperature est utilisée pour affecter la probabilité des caractères suivants.
  # Probabilité plus élevée == moins surprenante/ plus attendue
  # Une température plus basse == plus surprenante / moins attendue

  temperature = temp

  # Ici batch size == 1
  model.reset_states()

  for i in range(num_generate):

    # Générer des prédictions
    predictions = model(input_eval)

    # Supprimer la dimension de la forme du batch
    predictions = tf.squeeze(predictions, 0)

    # Utilisez une distribution catégorielle pour sélectionner le caractère suivant
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Passez le caractère prédit pour la prochaine entrée
    input_eval = tf.expand_dims([predicted_id], 0)

    # Transformer à nouveau en lettre de caractère
    text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))


In [49]:
print(generate_text(model, "flower", gen_size=1000))

flowers' sense
    And the unparrotake and bring your Highness- look between you, it
    were good to be knight; I had rather cry she hers.
  POLIXENES. He replies me too. Saying oppose in silence, Prince of my life, and sleeps on me defens the aful case hath not won thy worst with
    a vising frown.
  POLIXENES. That's as such as you must be friends;
    and as f that the world d doe to get forth,
    For then it hath true privilege of worth from her
    justifulate that have shall be in her life.
  AUFIDIUS. I am all the days less it I go away.
  MOTH. Sir Welch, sir, your legs are liming.
  ANTONIO. Do you see away the longer hove?
  DUKE. Dead, draw.
                    [GAINTE and PHOLUS
O Lo, what a speed? Is  
    had never may not grinvellou with actions.
  ANTONIO. Dost thou think, Desdemona is           Exit
  OTHELLO. Thou liest; thy name and Merculio. When we lov'd more.

  'MIDEA. That's not so.

                        Re-enter Troilus,
                    In open matter